<a href="https://colab.research.google.com/github/kirkwilson/crop_images/blob/main/crop_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

done
- user set crop variables
- upload zip of images

todo:
- unzip images
- crop images
- compress new images
- clean up old files
- write readme

compressed files

Input will likely be a compressed folder of images. Planning to support both zip
and 7z files. Will need to iterate through the `contents` folder and decompress.
If any file is a PNG or JPEG will move the file to a working folder. Working folder files will then be cropped.

In [ ]:
#@markdown (x, y) pixel coordinates of the top left crop rectangle.
crop_anchor_x = 7 #@param {type:"number"}
crop_anchor_y = 0 #@param {type:"number"}
#@markdown ---
#@markdown Crop area in pixels from `crop_anchor_y` down.
crop_height = 10 #@param {type:"number"}
#@markdown Crop area in pixels from `crop_anchor_x` right.
crop_width = 10 #@param {type:"number"}

In [ ]:
from collections import namedtuple
from google.colab import files
import os

def is_int(input) -> int:
  if not isinstance(input, int):
    raise TypeError(f'{input} is not an integer.')
  return input

def is_positive(input) -> int:
  if is_int(input):
    if input < 0:
      raise ValueError(f'{input} is not a positive number.')
  return input

def validate_input(input) -> int:
  is_int(input)
  is_positive(input)
  return input

def validate_coordinates(x, y) -> tuple[int, int]:
  validate_input(x)
  validate_input(y)
  return (x, y)

def get_point(x, y) -> tuple[int, int]:
  validate_coordinates(x, y)
  Point = namedtuple('Point', ['x', 'y'])
  return Point(x, y)

def upload_files() -> dict[str]:
  uploaded = files.upload()
  return uploaded.keys()

def is_existing_file(path) -> tuple[str, bool, bool]:
  IsExistingFile = namedtuple('ExistingFile', ['path', 'exists', 'is_file'])
  return IsExistingFile(path, os.path.exists(path), os.path.isfile(path))

def validate_is_existing_file(IsExistingFile) -> tuple[str, bool, bool]:
  if not IsExistingFile.path:
    error_message = ('No file path given. '
                     'A path is required when creating a valid IsExistingFile.')
    raise ValueError(error_message)
  if not IsExistingFile.exists:
    error_message = (f'Could not find "{IsExistingFile.path}". '
                     'An existing file is requried for a valid IsExistingFile.')
    raise FileNotFoundError(error_message)
  if not IsExistingFile.is_file:
    error_message = (f'"{IsExistingFile.path}" is not a file. '
                    'File required to create a valid IsExistingFile.')
    raise TypeError(error_message)
  return IsExistingFile

def is_valid_existing_file(path) -> tuple[str, bool, bool]:
  IsExistingFile = is_existing_file(path)
  validate_is_existing_file(IsExistingFile)
  return IsExistingFile

def split_basename(path) -> dict[dict[str, str]]:
  if is_valid_existing_file(path):
    filename, file_extension = os.path.splitext(path)
    file_dict = {
        path : {
            'filename': filename,
            'file_extension': file_extension}}
  return file_dict

In [ ]:
uploaded = upload_files()

files = {}
for upload in uploaded:
  ExistingFile = is_existing_file(upload)
  if all(ExistingFile):
    uploaded_file = split_basename(ExistingFile.path)
    files = files | uploaded_file

print(files)